In [2]:
import pandas as pd 
df = pd.read_excel("D:\IPL_Auction_Analytics\data\IPL_Bowler_Detailed_Data.xls")

df.head()


,Match_ID,Match_Date,Pitch_Type,Phase,Over,Ball,Bowler,Batter_Avg,Batter_SR,Runs_Conceded,Is_Wicket
0,29504,2023-04-12,Neutral,Powerplay,1,1,Bowler B,17.97,133.25,0,1
1,96402,2023-11-30,Batting,Powerplay,5,1,Bowler A,20.51,119.98,0,0
2,27383,2022-11-07,Neutral,Powerplay,1,5,Bowler B,30.05,124.73,2,0
3,99624,2022-04-10,Batting,Powerplay,5,3,Bowler B,21.29,147.69,1,0
4,65569,2023-06-23,Neutral,Powerplay,5,3,Bowler B,34.89,117.51,0,0


In [ ]:
df.columns

Index(['Match_ID', 'Match_Date', 'Pitch_Type', 'Phase', 'Over', 'Ball',
       'Bowler', 'Batter_Avg', 'Batter_SR', 'Runs_Conceded', 'Is_Wicket'],
      dtype='object')

In [7]:
df.shape

(4800, 11)

In [9]:
import pandas as pd

# Convert date to datetime format
df['Match_Date'] = pd.to_datetime(df['Match_Date'], errors='coerce')

# Ensure numeric columns are numeric
numeric_cols = ['Batter_Avg', 'Batter_SR', 'Runs_Conceded', 'Over', 'Ball']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Replace missing values if any
df.fillna({
    'Runs_Conceded': 0,
    'Is_Wicket': 0
}, inplace=True)

# Add helper columns
df['Runs_Conceded'] = df['Runs_Conceded'].astype(int)
df['Is_Wicket'] = df['Is_Wicket'].astype(int)

df.head()


,Match_ID,Match_Date,Pitch_Type,Phase,Over,Ball,Bowler,Batter_Avg,Batter_SR,Runs_Conceded,Is_Wicket
0,29504,2023-04-12,Neutral,Powerplay,1,1,Bowler B,17.97,133.25,0,1
1,96402,2023-11-30,Batting,Powerplay,5,1,Bowler A,20.51,119.98,0,0
2,27383,2022-11-07,Neutral,Powerplay,1,5,Bowler B,30.05,124.73,2,0
3,99624,2022-04-10,Batting,Powerplay,5,3,Bowler B,21.29,147.69,1,0
4,65569,2023-06-23,Neutral,Powerplay,5,3,Bowler B,34.89,117.51,0,0


In [10]:
bowler_group = df.groupby('Bowler').agg({
    'Runs_Conceded': 'sum',
    'Is_Wicket': 'sum',
    'Over': 'count',   # counts balls
    'Match_ID': 'nunique'
}).rename(columns={'Over': 'Balls_Bowled', 'Match_ID': 'Matches_Played'})

bowler_group.head()


,Runs_Conceded,Is_Wicket,Balls_Bowled,Matches_Played
Bowler,,,,
Bowler A,2935,108,2400,100
Bowler B,3654,231,2400,100


In [11]:
bowler_group['Overs_Bowled'] = bowler_group['Balls_Bowled'] / 6
bowler_group['Strike_Rate'] = bowler_group['Balls_Bowled'] / bowler_group['Is_Wicket'].replace(0, float('nan'))
bowler_group['Economy'] = bowler_group['Runs_Conceded'] / bowler_group['Overs_Bowled']
bowler_group['Bowling_Average'] = bowler_group['Runs_Conceded'] / bowler_group['Is_Wicket'].replace(0, float('nan'))

bowler_group.head()


,Runs_Conceded,Is_Wicket,Balls_Bowled,Matches_Played,Overs_Bowled,Strike_Rate,Economy,Bowling_Average
Bowler,,,,,,,,
Bowler A,2935,108,2400,100,400.0,22.222222,7.3375,27.175926
Bowler B,3654,231,2400,100,400.0,10.389610,9.1350,15.818182


In [12]:
def pressure_score(row):
    score = 0

    # Phase weights
    if row['Phase'] == 'Death':
        score += 30
    elif row['Phase'] == 'Powerplay':
        score += 15

    # Batter difficulty
    score += row['Batter_Avg'] * 0.1
    score += row['Batter_SR'] * 0.05

    # Pitch type
    if row['Pitch_Type'] == 'Batting':
        score += 10
    elif row['Pitch_Type'] == 'Bowling':
        score -= 10

    return score

df['Pressure'] = df.apply(pressure_score, axis=1)

pressure_metrics = df.groupby('Bowler')['Pressure'].mean().to_frame('Avg_Pressure_Faced')

pressure_metrics.head()


,Avg_Pressure_Faced
Bowler,
Bowler A,33.541655
Bowler B,33.852136
